In [1]:
# Used to pull data from Reddit
import praw
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import statsmodels.formula.api
import matplotlib.pyplot as plt
from datetime import datetime
import re


from gensim.models import Word2Vec
from nltk.corpus import brown
import nltk
import tensorflow as tf
from tensorflow.python.client import device_lib
import tensorflow.keras as keras 
from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

import tensorflow_hub as hub
import multiprocessing
from google.cloud import bigquery

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

#nltk.download('punkt')

/home/armand_kok/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
W0725 06:49:06.852599 140320478263104 deprecation_wrapper.py:119] From /home/armand_kok/anaconda3/lib/python3.7/site-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [2]:
#post_df.to_csv('reddit_posts_2016_2019.csv')
post_df = pd.read_csv('reddit_posts_2016_2019.csv', parse_dates=['created_dt_month'])

# Data PreProcessing

In [3]:
title_df = post_df[post_df['is_self'] == True][['title', 'subreddit']]
title_df.loc[title_df['subreddit'] == 'Conservative', 'political_leaning'] = 0
title_df.loc[title_df['subreddit'] == 'republicans', 'political_leaning'] = 0
title_df.loc[title_df['subreddit'] == 'democrats', 'political_leaning'] = 1
title_df.loc[title_df['subreddit'] == 'progressive', 'political_leaning'] = 1

# Neural Network Model

In [4]:
def replace_contraction(text):
    contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'can not'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text
def replace_links(text, filler=' '):
        text = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*',
                      filler, text).strip()
        return text
def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = replace_contraction(text)
    text = replace_links(text, "link")
    #text = remove_numbers(text)
    text = re.sub(r'[,!@#$%^&*)(|/><";:.?\'\\}{]',"",text)
    text = text.lower()
    return text


In [29]:
sample_df = title_df.sample(n=10000)[['title', 'political_leaning']]
sample_df['title_clean'] = sample_df['title'].apply(cleanText)
sample_df['word_token'] = sample_df['title_clean'].apply(nltk.word_tokenize)
sample_df['word_count'] = sample_df['word_token'].apply(lambda x: len(x))
sample_df = sample_df[sample_df['word_count']!=0].reset_index(drop=True)
X = np.array(sample_df['title_clean'])
y = np.array(sample_df['political_leaning'])

In [30]:
def ELMoEmbedding(x):
    embed = hub.Module("elmo_model")
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

def build_model(): 
    input_text = Input(shape=(1,), dtype="string")
    embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
    drop = keras.layers.Dropout(rate=0.5)(embedding)
    dense = Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(embedding)
    pred = Dense(1, activation='sigmoid')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

model_elmo = build_model()
model_elmo.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [33]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model_elmo.fit(X_train, y_train, epochs=10, batch_size=256, validation_split=0.2)

Train on 6396 samples, validate on 1600 samples
Epoch 1/10
6396/6396 [==============================] - 60s 9ms/sample - loss: 1.1338 - acc: 0.6565 - val_loss: 0.9277 - val_acc: 0.6725
Epoch 2/10
6396/6396 [==============================] - 46s 7ms/sample - loss: 0.8905 - acc: 0.6757 - val_loss: 0.8960 - val_acc: 0.6762
Epoch 3/10
6396/6396 [==============================] - 45s 7ms/sample - loss: 0.8394 - acc: 0.6793 - val_loss: 0.8107 - val_acc: 0.6744
Epoch 4/10
6396/6396 [==============================] - 45s 7ms/sample - loss: 0.7616 - acc: 0.7033 - val_loss: 0.7309 - val_acc: 0.6969
Epoch 5/10
6396/6396 [==============================] - 44s 7ms/sample - loss: 0.7268 - acc: 0.7022 - val_loss: 0.8366 - val_acc: 0.6725
Epoch 6/10
6396/6396 [==============================] - 43s 7ms/sample - loss: 0.7196 - acc: 0.6890 - val_loss: 0.7086 - val_acc: 0.6719
Epoch 7/10
6396/6396 [==============================] - 43s 7ms/sample - loss: 0.6989 - acc: 0.7025 - val_loss: 0.6820 - val_acc: 

In [34]:
with tf.Session() as session:

    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    model_elmo.evaluate(X_test, y_test)

  32/2000 [..............................] - ETA: 39s - loss: 1.1047 - acc: 0.4688

Exception ignored in: <function BaseSession._Callable.__del__ at 0x7f9e54619620>
Traceback (most recent call last):
  File "/home/armand_kok/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


2000/2000 [==============================] - 12s 6ms/sample - loss: 1.1116 - acc: 0.4970


In [36]:
1-y_test.sum()/y_test.shape[0]

0.6715

# Random Forest Model

In [36]:
def elmo_vectors(x):
    elmo_model = hub.Module('elmo_model', trainable=False)
    embeddings = elmo_model(x, signature="default", as_dict=True)['default']

    with tf.Session() as sess:
        sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
        return sess.run(embeddings)
    
X_list = [X[i:i+50] for i in range(0, X.shape[0], 50)]
X_vecs = [elmo_vectors(x) for x in X_list]
X_vecs = np.array(X_vecs)
X_vecs = X_vecs.reshape((X_vecs.shape[0] * X_vecs.shape[1], X_vecs.shape[2]))

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[50,512,51,45] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node module_37_apply_default/bilm/CNN/Conv2D_5 (defined at /home/armand_kok/anaconda3/lib/python3.7/site-packages/tensorflow_hub/native_module.py:561) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[module_37_apply_default/truediv/_3041]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[50,512,51,45] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node module_37_apply_default/bilm/CNN/Conv2D_5 (defined at /home/armand_kok/anaconda3/lib/python3.7/site-packag

In [ ]:
model = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X_vecs, y, test_size=0.2)

In [ ]:
model.fit(X_train, y_train)

In [21]:
print(metrics.classification_report(model.predict(X_train), y_train))
print(metrics.classification_report(model.predict(X_test), y_test))

             precision    recall  f1-score   support

        0.0       1.00      0.97      0.99       538
        1.0       0.95      1.00      0.97       262

avg / total       0.98      0.98      0.98       800

